# How to spinfoam amplitudes
## Computing the EPRL vertex amplitudes

In this notebook we show how to compute the EPRL vertices. 

We employ the [sl2cfoam-next](https://github.com/qg-cpt-marseille/sl2cfoam-next) library.
See also [paper describing the library](https://arxiv.org/abs/2107.13952).

In order to compile the cells below you first need to install the sl2cfoam-next. 

**Then "sl2c_data_folder" needs to be initialized with the appropriate path**, which on your system of course is different with respect
to the one written below. 

## Single vertex

We first focus on the the computation of a single amplitude

***The reference vertex amplitude for spin and intertwiner labels is the following:***

<img src="Pics/VertexAmplitude.png" width="500" />

Notice that the labels l inside the vertex indicate the spins not touched by the Y-map which need to be summed over. 

*This is already done "internally" by sl2cfoam-next*. Therefore in the following we can call the function with the external j labels

In [11]:
# required pkgs  
using HalfIntegers
using SL2Cfoam

# Barbero-Immirzi parameter
γ = 1.0

# initializing sl2cfoam-next   
sl2c_data_folder = "/home/frisus95/Scrivania/sl2cfoam_next/data_sl2cfoam"
sl2c_configuration = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0)
sl2c_result_return = (ret = true, store = true, store_batches = true)
# to store only the booster functions and not the full amplitude, use:
# sl2c_result_return = (ret = true, store = false, store_batches = false);
# for further options, see the library documentation
SL2Cfoam.cinit(sl2c_data_folder, γ, sl2c_configuration) 

# homogeneous cut-off 
Δl = 15;

### Full vertex

We compute the EPRL vertex amplitude for each possible value of the boundary intertwiners

In [3]:
# set the 10 vertex boundary spins
spins = j245, j125, j124, j145, j235, j234, j345, j123, j135, j134 = [1 for _ = 1:10]

@time v = vertex_compute(spins, Δl; result = sl2c_result_return);

# v.a is the 5-dimensional array with the data
vertex = v.a

  0.000050 seconds (7 allocations: 400 bytes)


3×3×3×3×3 Array{Float64, 5}:
[:, :, 1, 1, 1] =
  5.75025e-10  -7.78342e-10  1.13462e-9
 -7.81249e-10   6.07616e-10  8.14858e-10
  1.13721e-9    8.1391e-10   2.84652e-10

[:, :, 2, 1, 1] =
 -7.78342e-10   6.04668e-10  8.10461e-10
  1.14875e-9   -4.32425e-10  6.22799e-10
 -1.59089e-9   -6.0113e-10   1.31907e-10

[:, :, 3, 1, 1] =
  1.13462e-9   8.10461e-10  2.82626e-10
 -1.59389e-9  -6.01495e-10  1.31614e-10
  2.23773e-9  -8.22919e-10  1.09464e-10

[:, :, 1, 2, 1] =
 -7.81249e-10   1.14875e-9  -1.59389e-9
  1.14603e-9   -1.06916e-9  -1.32773e-9
 -1.59132e-9   -1.3286e-9   -4.67295e-10

[:, :, 2, 2, 1] =
  6.07616e-10  -4.32425e-10  -6.01495e-10
 -1.06916e-9    4.38984e-10  -6.55779e-10
  1.34991e-9    5.52239e-10  -1.12279e-10

[:, :, 3, 2, 1] =
  8.14858e-10   6.22799e-10  1.31614e-10
 -1.32773e-9   -6.55779e-10  7.90686e-12
  1.7698e-9    -7.89167e-10  1.12574e-10

[:, :, 1, 3, 1] =
  1.13721e-9  -1.59089e-9  2.23773e-9
 -1.59132e-9   1.34991e-9  1.7698e-9
  2.2381e-9    1.76957e-9  6.

### Range for boundary intertwiners

We can also compute the EPRL vertex amplitude for a restricted range of boundary intertwiners.

In [8]:
# homogeneous cut-off 
Δl = 0

# initialize each boundary spins with a random integer value between 1 and 3
# incrementing the range, one has to pay attention to triangular inequalities
spins = j245, j125, j124, j145, j235, j234, j345, j123, j135, j134 = [rand(1:3) for _ = 1:10]

@show spins

# determine the range of boundary intertwiners
r1245, _ = intertwiner_range(j245, j125, j124, j145)
r2345, _ = intertwiner_range(j235, j234, j345, j245) 
r1235, _ = intertwiner_range(j123, j135, j125, j235)
r1234, _ = intertwiner_range(j134, j124, j234, j123)
r1345, _ = intertwiner_range(j145, j345, j135, j134)
    
# intertwiners fixed to their minimum value
i1245 =  r1245[1]  
i2345 =  r2345[1]   
i1345 =  r1345[1]   

# restricted range of boundary intertwiners to compute
vertex_intertw_ranges = ((i1245,i1245), (i2345, i2345), r1235, r1234, (i1345,i1345))

@show vertex_intertw_ranges

@time v = vertex_compute(spins, Δl, vertex_intertw_ranges; result = sl2c_result_return);

vertex = v.a

spins = [2, 3, 1, 3, 2, 3, 1, 2, 3, 1]
vertex_intertw_ranges = ((2, 2), (1, 1), (1, 5), (1, 2), (2, 2))
  0.011214 seconds (7 allocations: 400 bytes)


1×2×5×1×1 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 9.88921e-16  2.95345e-14

[:, :, 2, 1, 1] =
 2.18095e-14  -1.54102e-13

[:, :, 3, 1, 1] =
 -1.79719e-13  5.03032e-13

[:, :, 4, 1, 1] =
 7.33604e-13  -6.88867e-13

[:, :, 5, 1, 1] =
 -2.55182e-12  -2.2469e-12

## Vertices for the Δ4 amplitude

We now compute and store all EPRL vertices required for the computation of the Δ4 amplitude.

The vertices are computed for increasing values of the cut-off Δl.

For simplicity we set all boundary spins equal to j, computing the full vertices.

<img src="Pics/Delta_4.SVG" alt="Drawing" style="width: 700px;"/>

In [5]:
# required pkgs
using JLD2   
using DelimitedFiles
using Dates

# current folder
root_dir = pwd()

# print with flush
function log(x...)
println(now(), " ", join(x, " ")...)
flush(stdout)
end

# maximal cut-off  
Δl_max = 15;

In [5]:
# value of all boundary spins 
j = 1.0

for Δl = 0:Δl_max
    
   # folder with vertex amplitudes 
   vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                 
   mkpath(vertex_path)
    
   # if all boundary spins are integers and equal to j, bulk spin has range [0, 3j]   
   j_bulk_min, j_bulk_max = 0, 3j
    
   for j_bulk = j_bulk_min:j_bulk_max
    
      v = vertex_compute([j, j, j, j, j, j, j, j_bulk, j, j], Δl; result = sl2c_result_return);        
      log("computed full vertex tensor for Immirzi=$(γ) j_boundary=$(j) j_bulk=$(j_bulk) Dl=$(Δl)")
                
      vertex = v.a;        
      @save "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2" vertex         
               
   end # bulk spin cycle
      
end # Δl cycle

2022-01-28T14:18:00.007 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=0.0 Dl=0
2022-01-28T14:18:02.653 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=1.0 Dl=0
2022-01-28T14:18:02.656 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=2.0 Dl=0
2022-01-28T14:18:02.658 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=3.0 Dl=0
2022-01-28T14:18:02.660 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=0.0 Dl=1
2022-01-28T14:18:02.661 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=1.0 Dl=1
2022-01-28T14:18:02.663 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=2.0 Dl=1
2022-01-28T14:18:02.664 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=3.0 Dl=1
2022-01-28T14:18:02.666 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=0.0 Dl=2
2022-01-28T14:18:02.668 computed full vertex tensor for Immirzi=1.0 j_boundary=1.0 j_bulk=1.0 Dl=2
2022-01-28

For convenience, we write a function that computes all the required full vertices for given lists of boundary parameters in the symmetric configuration

In [6]:
function initializing_library(Immirzi)
   
folder = "/home/frisus95/Scrivania/sl2cfoam_next/data_sl2cfoam";
conf = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0);
SL2Cfoam.cinit(folder, Immirzi, conf);
    
end    

function Δ4_vertices(Immirzi_list, j_boundary_list, Δl_max)

for γ in Immirzi_list
    
   initializing_library(γ)    

   for j in j_boundary_list   
    
       for Δl = 0:Δl_max
        
             vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                 
             mkpath(vertex_path)
                
             if (iszero(j%1)) j_bulk_min = 0 else j_bulk_min = 0.5 end       
             j_bulk_max = 3j
    
             for j_bulk = j_bulk_min:j_bulk_max
    
                v = vertex_compute([j, j, j, j, j, j, j, j_bulk, j, j], Δl; result = sl2c_result_return);        
                log("computed full vertex tensor for Immirzi=$(γ) j_boundary=$(j) j_bulk=$(j_bulk) Δl=$(Δl)")
                
                vertex = v.a;        
                @save "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2" vertex         
               
             end # bulk spin cycle
      
        end # cut-off cycle
            
    end # j_boundary cycle
    
end  # Immirzi cycle
    
end

Δ4_vertices (generic function with 1 method)

In [7]:
# list of Immirzi parameters 
Immirzi_list = [1.0]

# list of boundary spins (symmetric configurations)
j_boundary_list = [0.5]

# maximal cut-off
Δl_max = 15;

In [9]:
Δ4_vertices(Immirzi_list, j_boundary_list, Δl_max)

2022-01-28T14:21:33.413 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=0.5 Δl=0
2022-01-28T14:21:33.450 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=1.5 Δl=0
2022-01-28T14:21:33.454 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=0.5 Δl=1
2022-01-28T14:21:33.458 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=1.5 Δl=1
2022-01-28T14:21:33.462 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=0.5 Δl=2
2022-01-28T14:21:33.466 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=1.5 Δl=2
2022-01-28T14:21:33.470 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=0.5 Δl=3
2022-01-28T14:21:33.474 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=1.5 Δl=3
2022-01-28T14:21:33.477 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=0.5 Δl=4
2022-01-28T14:21:33.480 computed full vertex tensor for Immirzi=1.0 j_boundary=0.5 j_bulk=1.5 Δl=4
2022-01-28